In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb
import requests
from bs4 import BeautifulSoup
import time
import unicodedata

In [80]:
top_df = pd.read_csv('top.csv') 
second_merge_df = pd.read_csv('second_merge.csv') 

In [83]:
merged_df = duckdb.sql("""SELECT top_df.*, 
                            second_merge_df.valence AS valence_second, 
                            second_merge_df.danceability AS danceability_second, 
                            second_merge_df.energy AS energy_second, 
                            second_merge_df.tempo AS tempo_second
                        FROM top_df 
                        LEFT JOIN second_merge_df
                        ON 
                            top_df.Year = second_merge_df.year AND 
                            top_df.Rank = second_merge_df.Rank AND 
                            top_df.Artist = second_merge_df.Artist AND 
                            top_df.Song = second_merge_df.Song
                        ORDER BY top_df.Year, top_df.Rank 
                        """).df() 


merged_df['valence'] = merged_df['valence'].fillna(merged_df['valence_second'])
merged_df['danceability'] = merged_df['danceability'].fillna(merged_df['danceability_second'])
merged_df['energy'] = merged_df['energy'].fillna(merged_df['energy_second'])
merged_df['tempo'] = merged_df['tempo'].fillna(merged_df['tempo_second'])

merged_df.drop(columns=['valence_second', 'danceability_second', 'energy_second', 'tempo_second'], inplace=True)

no_match = merged_df[merged_df['valence'].isna()]

# print(merged_df) 

print(no_match) 


merged_df.to_csv('top_music.csv', index=False)


      Year  Rank                          Artist  \
3     1960     4                  johnny preston   
15    1960    16               hollywood argyles   
26    1960    27  brook benton  dinah washington   
39    1960    40                  freddie cannon   
48    1960    49   hank ballard  the midnighters   
...    ...   ...                             ...   
5568  2016    69            yo gotti nicki minaj   
5584  2016    85        major lazer  dj snake mo   
5585  2016    86    shawn mendes  camila cabello   
5586  2016    87                     fat joe rem   
5593  2016    94     hailee steinfeld  grey zedd   

                                 Song  valence  danceability  energy  tempo  
3                        running bear      NaN           NaN     NaN    NaN  
15                           alleyoop      NaN           NaN     NaN    NaN  
26                               baby      NaN           NaN     NaN    NaN  
39     way down yonder in new orleans      NaN           NaN   

In [117]:
clean = merged_df
clean.shape

(5600, 8)

In [118]:
clean.head()

,Year,Rank,Artist,Song,valence,danceability,energy,tempo
0,1960,1,percy faith,theme from a summer place,0.9130,0.325,0.4120,186.720
1,1960,2,jim reeves,hell have to go,0.1905,0.533,0.2435,81.068
2,1960,3,everly brothers,cathys clown,0.8660,0.498,0.5820,119.809
3,1960,4,johnny preston,running bear,NaN,NaN,NaN,NaN
4,1960,5,mark dinning,teen angel,0.3300,0.447,0.1820,95.094


In [119]:
clean = clean.dropna(subset='valence')

In [120]:
clean= clean.dropna(subset='danceability')

In [122]:
countyear= duckdb.sql("""SELECT Year, COUNT(year)
                        FROM clean 
                        GROUP By Year 
                        ORDER BY Year""").df()

In [123]:
countyear 

,Year,"count(""year"")"
0,1960,83
1,1961,76
2,1962,76
3,1963,81
4,1964,42
5,1965,51
6,1966,49
7,1967,51
8,1968,54
9,1969,52
